# Kleven Best (2012) Replication

In [ ]:
# Load libraries
using Parameters
using CSV
using DataFrames
using Plots
using Trapz
using NumericalIntegration
using SpecialFunctions

In [ ]:
# (Some imports / temporary)
# using Pkg
# Pkg.add("SpecialFunctions")

## Data setup

## Optimal tax calculation

### Procedure

In [ ]:
# Set parameters
# Set some parameters
@with_kw struct params
    γ ::Int64            # SWF inequality aversion parameter
    k ::Int64            # Parametrizes elasticity ε
    e ::Float64 = 1/k    # Elasticity ε = 1/k
    a ::Int64            # Pareto parameter for upper income distribution
    R ::Int64            # Exogenous per-person revenue requirement
end;